In [783]:
import pandas as pd
from datetime import date, datetime, timedelta
import pickle
import numpy as np
from math import sqrt
import copy
import matplotlib.pyplot as plt
%matplotlib inline
import math

<h3> Hotspot calculation (Also used in GUI) <h3>

In [784]:
def open_file():
    try:
        data_file = pd.read_csv(DEFAULT_FILE_PATH + CSV_FILE_EXTENSION)
    except:
        print("File not found")
        return pd.DataFrame()
    return data_file

In [785]:
# Define file variables
DEFAULT_FILE_PATH = "CrimeGUI/data"
CSV_FILE_EXTENSION = ".csv"
DATA_FILE = pd.DataFrame()
FILTERED_DATA = pd.DataFrame()
DATA_FILE = open_file()
FIRST_DATE = datetime.strptime(DATA_FILE['Date'].iloc[0], '%d/%m/%Y')
LAST_DATE = datetime.strptime(DATA_FILE['Date'].iloc[len(DATA_FILE) - 1], '%d/%m/%Y')
MODEL_FEATURE_SEPARATOR = "_"
MODEL_PATH = "CrimeGUI/Models/"
COMMA_SPACE = ", "
INCIDENT_COL_KEY = "Todays Reports"
NEIGHBOURHOOD_COL_KEY = "Neighborhood"
DATE_COL_KEY = "Date"

In [786]:
# Define feature selection variables
F_REGRESSION_NAME = "F-Regression"
F_REGRESSION_FILE_TAG = "f_regression"
CHI2_NAME = "Chi-Squared"
CHI2_FILE_TAG = "chi2"
ADABOOST_NAME = "AdaBoost"
ADABOOST_FILE_TAG = "adaboost"
EQUAL_DATA_NAME = "Equal Selection"
EQUAL_DATA_FILE_TAG = "equal_crime_and_business"
ALL_BUS_NAME = "All Business"
ALL_BUS_FILE_TAG = "all_business"
FEATURE_SELECTION = [F_REGRESSION_NAME, CHI2_NAME, ADABOOST_NAME, EQUAL_DATA_NAME, ALL_BUS_NAME]
FEATURES = {
    F_REGRESSION_NAME : ['Reports 1 day ago', 'Reports 2 days ago', 'Reports 3 days ago',
                       'Reports 4 days ago', 'Reports 5 days ago', 'Reports 6 days ago',
                      'Reports 7 days ago','Reports 14 days ago','Reports 30 days ago','Reports 365 days ago'],
    CHI2_NAME : ['South of Market', 'Mission', 'Tenderloin', 'Number of businesses', 
               'Downtown / Union Square', 'Civic Center', 'Reports 365 days ago',
               'Reports 1 day ago','Reports 2 days ago','Reports 14 days ago'],
    ADABOOST_NAME : ['Reports 365 days ago', 'Reports 1 day ago', 'Reports 14 days ago', 'Reports 3 days ago', 
               'Reports 2 days ago', 'Reports 7 days ago', 'Number of businesses',
               'Reports 4 days ago','Reports 5 days ago','Closures 365 days ago'],
    EQUAL_DATA_NAME : ['Number of businesses', 'Last 28 days closures', 'Last 7 days openings',
                          'Last 14 days closures', 'Last 7 days closures','Reports 1 day ago',
                      'Reports 2 days ago', 'Reports 4 days ago', 'Reports 30 days ago', 'Reports 7 days ago'],
    ALL_BUS_NAME : ['Number of businesses', 'Last 28 days closures', 'Last 7 days openings',
                          'Last 14 days closures', 'Last 7 days closures','Number of openings',
                   'Openings 4 days ago','Openings 1 day ago', 'Openings 7 days ago', 'Openings 2 days ago']
    }
FEATURE_FILE_TAGS = {
    F_REGRESSION_NAME : F_REGRESSION_FILE_TAG,
    CHI2_NAME : CHI2_FILE_TAG,
    ADABOOST_NAME : ADABOOST_FILE_TAG,
    EQUAL_DATA_NAME : EQUAL_DATA_FILE_TAG,
    ALL_BUS_NAME : ALL_BUS_FILE_TAG
    }
FEATURE_NAMES_BY_FILE_TAG = {
    F_REGRESSION_FILE_TAG : F_REGRESSION_NAME,
    CHI2_FILE_TAG : CHI2_NAME,
    ADABOOST_FILE_TAG : ADABOOST_NAME,
    EQUAL_DATA_FILE_TAG : EQUAL_DATA_NAME,
    ALL_BUS_FILE_TAG : ALL_BUS_NAME
}

In [787]:
# Define model variables
ANN_NAME = "Multi-Layer\nPerceptron"
ANN_FILE_TAG = "multi_layer_perceptron"
DECISION_TREE_NAME = "Decision Tree"
DECISION_TREE_FILE_TAG = "decision_tree"
ELASTIC_NET_NAME = "Elastic Net"
ELASTIC_NET_FILE_TAG = "elastic_net"
LASSO_NAME = "Lasso"
LASSO_FILE_TAG = "lasso"
LINERAR_REGRESSION_NAME = "Linear \nRegression"
LINERAR_REGRESSION_FILE_TAG = "linear_regression"
RANDOM_FOREST_NAME = "Random \nForest"
RANDOM_FOREST_FILE_TAG = "random_forest"
RIDGE_REGRESSION_NAME = "Ridge \nRegression"
RIDGE_REGRESSION_FILE_TAG = "ridge_regression"
SVM_NAME = "SVM"
SVM_FILE_TAG = "svm"
MODELS = [ANN_NAME,
          DECISION_TREE_NAME,
          ELASTIC_NET_NAME,
          LASSO_NAME,
          LINERAR_REGRESSION_NAME,
          RANDOM_FOREST_NAME,
          RIDGE_REGRESSION_NAME,
          SVM_NAME]
MODEL_FILE_TAGS = {
    ANN_NAME : ANN_FILE_TAG,
    DECISION_TREE_NAME : DECISION_TREE_FILE_TAG,
    ELASTIC_NET_NAME : ELASTIC_NET_FILE_TAG,
    LASSO_NAME : LASSO_FILE_TAG,
    LINERAR_REGRESSION_NAME: LINERAR_REGRESSION_FILE_TAG,
    RANDOM_FOREST_NAME: RANDOM_FOREST_FILE_TAG,
    RIDGE_REGRESSION_NAME: RIDGE_REGRESSION_FILE_TAG,
    SVM_NAME : SVM_FILE_TAG
    }
MODEL_NAMES_BY_FILE_TAG = {
    ANN_FILE_TAG : ANN_NAME,
    DECISION_TREE_FILE_TAG : DECISION_TREE_NAME,
    ELASTIC_NET_FILE_TAG : ELASTIC_NET_NAME,
    LASSO_FILE_TAG : LASSO_NAME,
    LINERAR_REGRESSION_FILE_TAG : LINERAR_REGRESSION_NAME,
    RANDOM_FOREST_FILE_TAG: RANDOM_FOREST_NAME,
    RIDGE_REGRESSION_FILE_TAG : RIDGE_REGRESSION_NAME,
    SVM_FILE_TAG : SVM_NAME
}

In [788]:
PARENT_NEIGHBOURHOODS = {
    "Central Waterfront" : ["Dogpatch"],
    "Eureka Valley" : ["Dolores Heights","Castro"],
    "Buena Vista" : ["Ashbury Heights"],
    "Cole Valley" : ["Parnassus Heights"],
    "Bayview" : ["Apparel City", "Produce Market"],
    "Russian Hill" : ["Aquatic Park / Ft. Mason"],
    "North Beach" : ["Bret Harte"],
    "Western Addition" : ["Cathedral Hill", "Japantown"],
    "Downtown / Union Square" : ["Fairmount", "Chinatown", "Lower Nob Hill", "Polk Gulch"],
    "Mission Terrace" : ["Cayuga"],
    "Northern Waterfront" : ["Fishermans Wharf"],
    "Bernal Heights" : ["Holly Park", "Peralta Heights", "St. Marys Park"],
    "Hunters Point" : ["India Basin"],
    "Forest Hill" : ["Laguna Honda"],
    "Hayes Valley" : ["Lower Haight"],
    "Portola" : ["McLaren Park", "University Mound"],
    "South of Market" : ["Mint Hill"],
    "Stonestown" : ["Parkmerced"],
    "Presidio Heights" : ["Presidio Terrace"],
    "South Beach" : ["Rincon Hill"],
    "Potrero Hill" : ["Showplace Square"],
    "Visitacion Valley" : ["Sunnydale"],
    "Lincoln Park / Ft. Miley" : ["Sutro Heights"],
    "Cow Hollow" : ["Union Street"]
    }

In [789]:
def get_hotspots(data, model_key, features_key,calendar_date):
    x_data = data.loc[data[DATE_COL_KEY].str.contains(calendar_date)]
    neighbourhoods_data = pd.DataFrame(x_data[NEIGHBOURHOOD_COL_KEY])
    neighbourhoods_data.reset_index(drop=True, inplace=True)
    y_data = pd.DataFrame(x_data[INCIDENT_COL_KEY])
    y_data.reset_index(drop=True, inplace=True)
    features_selected = FEATURES[features_key]
    x_data = x_data[features_selected]
    return load_model(x_data, y_data, neighbourhoods_data, model_key, features_key)

In [790]:
def load_model(x_data, y_data, neighbourhoods_data, model_key, features_key):
    model_tag = MODEL_FILE_TAGS[model_key]
    feature_tag = FEATURE_FILE_TAGS[features_key]
    file_path = MODEL_PATH + model_tag + MODEL_FEATURE_SEPARATOR + feature_tag
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
        return make_prediction(model, x_data, y_data, neighbourhoods_data)

In [791]:
def make_prediction(model, x_data, y_data, neighbourhoods_data):
    y_predict = model.predict(x_data)
    y_actual,y_predict,neighbourhoods_data = merge_sub_neighbourhoods(y_data,y_predict,neighbourhoods_data)
    total_predictions = len(y_predict)
    neighbourhoods_data.reset_index(drop=True, inplace=True)
    prediction_neighbourhoods = neighbourhoods_data[NEIGHBOURHOOD_COL_KEY].to_numpy()
    actual_neighbourhoods = prediction_neighbourhoods.copy()
    indexes = y_actual.argsort()
    y_actual = np.flip(y_actual[indexes])
    actual_neighbourhoods = np.flip(actual_neighbourhoods[indexes])
    indexes = y_predict.argsort()
    y_predict = np.flip(y_predict[indexes])
    prediction_neighbourhoods = np.flip(prediction_neighbourhoods[indexes])
    return y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods

In [792]:
def get_non_negative_value(value):
    if value < 0:
        return 0
    else:
        return value

In [793]:
def merge_sub_neighbourhoods(y_data,y_predict,neighbourhoods_data):
    y_data = pd.DataFrame(y_data).to_numpy().flatten()
    indexes_to_remove = []
    for parent_key in PARENT_NEIGHBOURHOODS:
        parent_index = neighbourhoods_data.index[neighbourhoods_data[NEIGHBOURHOOD_COL_KEY] == parent_key].tolist()[0]
        y_predict_parent_value = get_non_negative_value(y_predict[parent_index])
        y_data_parent_value = get_non_negative_value(y_data[parent_index])
        for sub_neighbourhood in PARENT_NEIGHBOURHOODS[parent_key]:
            sub_neighbourhood_index = neighbourhoods_data.index[neighbourhoods_data[NEIGHBOURHOOD_COL_KEY] == sub_neighbourhood].tolist()[0]
            indexes_to_remove.append(sub_neighbourhood_index)
            y_predict_sub_neighbourhood_value = get_non_negative_value(y_predict[sub_neighbourhood_index])
            y_predict_parent_value = y_predict_parent_value + y_predict_sub_neighbourhood_value
            y_data_sub_neighbourhood_value = get_non_negative_value(y_data[sub_neighbourhood_index])
            y_data_parent_value = y_data_parent_value + y_data_sub_neighbourhood_value
        y_predict[parent_index] = y_predict_parent_value
        y_data[parent_index] = y_data_parent_value
    neighbourhoods_data = neighbourhoods_data.drop(neighbourhoods_data.index[indexes_to_remove])
    indexes_to_remove.sort(reverse=True)
    for index in indexes_to_remove:
        y_predict = np.delete(y_predict,index)
        y_data = np.delete(y_data,index)
    return y_data,y_predict,neighbourhoods_data 

<h3>Evaluation <h3>

In [853]:
def calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods,num_hotspots):
    total_predictions = len(y_predict)
    predicted_hotspots, actual_hotspots = determine_hotspots(y_actual, 
                                                             actual_neighbourhoods, 
                                                             prediction_neighbourhoods,
                                                             num_hotspots)
    num_predictions = len(predicted_hotspots)
    classification_scores = calculate_standard_scores(num_predictions,
                                                      predicted_hotspots,
                                                      actual_hotspots,
                                                      total_predictions)
    misclassification_scores = get_missed_incidents(predicted_hotspots, 
                                                    actual_hotspots, 
                                                    y_actual, 
                                                    y_predict,
                                                    num_predictions,
                                                    actual_neighbourhoods)
    return classification_scores + misclassification_scores

In [1260]:
def get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict,num_predictions,actual_neighbourhoods):
    incidents_correct = 0
    incidents_missed = 0
    additional_incidents_caught = 0
    for neighbourhood in predicted_hotspots:
        index = np.where(actual_neighbourhoods == neighbourhood)[0]
        value = y_actual[index]
        if neighbourhood in actual_hotspots:
            incidents_correct += value
        else:
            additional_incidents_caught += value
    i = 0
    i_limit = 0
    lowest_actual = y_actual[num_predictions-1]
    while i < num_predictions:
        if y_actual[i] != lowest_actual:
            neighbourhood = actual_hotspots[i]
            if neighbourhood not in predicted_hotspots:
                index = np.where(actual_neighbourhoods == neighbourhood)[0]
                value = y_actual[index]
                incidents_missed += value
            i += 1
        else:
            i_limit = i
            i = num_predictions
    i = i_limit
    remaining_hotspots_to_find = num_predictions - i
    hotspots_found = 0
    while i < len(actual_hotspots):
        neighbourhood = actual_hotspots[i]
        if neighbourhood in predicted_hotspots:
            hotspots_found += 1
        i += 1
    num_lowest_value_hotspots_missing = remaining_hotspots_to_find - hotspots_found
    lowest_values_missed = lowest_actual * num_lowest_value_hotspots_missing
    incidents_missed += lowest_values_missed
    total_actual_hotspots = incidents_correct + incidents_missed
    net_missed = incidents_missed - additional_incidents_caught
    misclassification_severity = 0.0
    if net_missed != 0:
        misclassification_severity = (net_missed / total_actual_hotspots)[0]
    return [misclassification_severity]

In [796]:
def determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots):
    #sorted numpy arrays expected 
    predicted_hotspots = prediction_neighbourhoods[:num_hotspots]
    actual_hotspots = actual_neighbourhoods[:num_hotspots]
    lowest_hotspot_value = y_actual[num_hotspots-1]
    i = num_hotspots
    while i < len(actual_neighbourhoods):
        if y_actual[i] == lowest_hotspot_value:
            np.append(actual_hotspots,actual_neighbourhoods[i])
            i += 1
        else:
            i = len(actual_neighbourhoods)
    return predicted_hotspots, actual_hotspots

In [1073]:
def calculate_standard_scores(num_predictions,predicted_hotspots,actual_hotspots,total_predictions):
    #true positives
    tp = 0
    #false positives
    fp = 0
    for i in range(num_predictions):
        if predicted_hotspots[i] in actual_hotspots:
            #print(str(predicted_hotspots[i]) + " found,")
            tp += 1
        else:
            fp +=1
            #print(str(predicted_hotspots[i]) + " not found,")
    #true negatives
    tn = total_predictions-num_predictions-fp
    #false negatives
    fn = fp
    #sensitivity/recall
    sensitivity = 0
    if tp + fn != 0:
        sensitivity = tp / (tp + fn)
    #specificity
    specificity = 1
    if tn + fp !=0:
        specificity = tn / (tn + fp)
    #precision
    precision = 0
    if tp + fp !=0:
        precision = tp / (tp + fp)
    #f1 - incorporates both sensitivity/recall and precision
    f1 = 0
    if precision + sensitivity != 0:
        f1 = round(2 * (precision * sensitivity) / (precision + sensitivity),5)
    #matthews correlation coefficient
    mcc = 1
    if sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn+fn)) != 0:
        mcc = round(((tp * tn) - (fp * fn)) / (sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn+fn))),5)
    return [sensitivity, specificity, precision, f1, mcc]

In [798]:
def get_results(num_hotspots):
    all_scores = {}
    for score_key in SCORE_KEYS:
        all_scores[score_key] = []
    algorithm_names = []
    for model_key in MODELS:
        print(model_key)
        for feature_key in FEATURES:
            print(feature_key)
            scores = score_algorithm(model_key, feature_key,num_hotspots)
            for score_key in SCORE_KEYS:
                all_scores[score_key].append(scores[score_key])
            algorithm_names.append(model_key + "\n" + feature_key)
    return all_scores, algorithm_names

In [799]:
def score_algorithm(model_key, feature_key,num_hotspots):
    start_date = FIRST_DATE
    all_scores = {}
    for score_key in SCORE_KEYS:
        all_scores[score_key] = []
    while start_date <= LAST_DATE:
        y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods = get_hotspots(DATA_FILE, 
                                                                                model_key, 
                                                                                feature_key, 
                                                                                str(start_date.strftime("%d/%m/%Y")))
        results = calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods,num_hotspots)
        for i in range(len(SCORE_KEYS)):
            all_scores[SCORE_KEYS[i]].append(results[i])
        start_date += timedelta(days=1)
    averages = {}
    for score_key in SCORE_KEYS:
        averages[score_key] = sum(all_scores[score_key]) / len (all_scores[score_key])
    return averages

In [800]:
def best_x_scores(score_metric,x,algorithms,results,num_hotspots):
    all_labels = algorithms.copy()
    x_labels = []
    best_scores = []
    scores = results[score_metric].copy()
    num_scores = len(scores)
    x = validate_x(num_scores,x)
    i = 0
    while i < x:
        if HIGHEST_IS_BEST[score_metric]:
            index = np.argmax(scores)
        else:
            index = np.argmin(scores)
        x_labels.append(all_labels[index])
        best_scores.append(scores[index])
        all_labels = np.delete(all_labels,index)
        scores = np.delete(scores,index)
        i+=1
    y_select = np.arange(len(best_scores))
    plt.figure(figsize = (15,15))
    plt.xticks(y_select,x_labels)
    #plt.xticks(rotation=45)
    plt.title("Top " + str(x) + " " + score_metric + " scores\nwhen predicting the top " 
              + str(num_hotspots) + " Crime Hotspots")
    plt.xlabel("Algorithm")
    plt.ylabel(score_metric + " score")
    high_score = max(best_scores)
    low_score = min(best_scores)
    min_y = low_score - 0.5 * (high_score - low_score)
    max_y = high_score + 0.5 * (high_score - low_score)
    plt.ylim(bottom = min_y, top = max_y)
    bar = plt.bar(y_select, best_scores)
    return bar

In [801]:
def get_labels():
    labels = []
    for algorithm in algorithms:
        print(algorithm)
        for sel_method in sel_methods:
            print(sel_method)
            labels.append(algorithm_display_names[algorithms.index(algorithm)] + "\nusing\n" + 
                          feature_select_display_names[np.nonzero(sel_methods == sel_method)[0][0]] + "\ndataset")
    return labels

In [802]:
def validate_x(num_scores,x):
    soft_cap = 15
    if x > num_scores or x < 1:
        if num_scores > soft_cap:
            return soft_cap
        else:
            return num_scores
    return x

In [803]:
def evaluate_models(num_hotspots_range, top_x):
    bars = []
    for i in num_hotspots_range:
        print(i)
        num_hotspots = i
        results, algorithms = get_results(num_hotspots)
        for metric in SCORE_KEYS:
            bars.append(best_x_scores(metric,top_x,algorithms,results,num_hotspots))
    return bars

In [804]:
SENSITIVITY_KEY = "Sensitivity"
SPECIFICITY_KEY = "Specificity"
PRECISION_KEY = "Precision"
F1_KEY = "F1"
MCC_KEY = "MCC"
SEVERITY_KEY = "Lowest Misclassification Severity"
SCORE_KEYS = [
    SENSITIVITY_KEY,
    SPECIFICITY_KEY,
    PRECISION_KEY,
    F1_KEY,
    MCC_KEY,
    SEVERITY_KEY
]
HIGHEST_IS_BEST = {
    SENSITIVITY_KEY: True,
    SPECIFICITY_KEY: True,
    PRECISION_KEY: True,
    F1_KEY:True,
    MCC_KEY: True,
    SEVERITY_KEY: False
}

num_hotspots_range = [5,10,15,20]    #[5,10,15,20]

In [408]:
top_x = 10
bars = evaluate_models(num_hotspots_range,top_x)

5
Multi-Layer
Perceptron
F-Regression
['South of Market' 'Mission' 'Downtown / Union Square' 'Tenderloin'
 'Western Addition']
['Downtown / Union Square' 'South of Market' 'Tenderloin' 'Bayview'
 'Bernal Heights']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Western Addition']
['South of Market' 'Outer Sunset' 'Western Addition' 'Duboce Triangle'
 'Bayview']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Bayview']
['South of Market' 'Downtown / Union Square' 'Mission' 'Potrero Hill'
 'Civic Center']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Potrero Hill']
['Mission' 'Downtown / Union Square' 'Civic Center' 'South of Market'
 'Northern Waterfront']
['Downtown / Union Square' 'Mission' 'South of Market' 'Tenderloin'
 'Bayview']
['South of Market' 'Tenderloin' 'Bayview' 'Downtown / Union Square'
 'Mission']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Bayview']
['South of Market' 'Downtown

['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Civic Center']
['Downtown / Union Square' 'South of Market' 'North Beach'
 'Mission Terrace' 'Civic Center']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Western Addition']
['Downtown / Union Square' 'Mission' 'Tenderloin' 'Marina' 'Parkside']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Western Addition']
['Mission' 'Potrero Hill' 'Downtown / Union Square' 'South of Market'
 'Bayview']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Potrero Hill']
['South of Market' 'Tenderloin' 'Downtown / Union Square' 'Mission'
 'Western Addition']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Potrero Hill']
['Downtown / Union Square' 'South of Market' 'Outer Richmond' 'Tenderloin'
 'Mission']
['Downtown / Union Square' 'South of Market' 'Mission' 'Tenderloin'
 'Potrero Hill']
['South of Market' 'Tenderloin' 'Mission' 'Civic Center' 'B

['South of Market' 'Downtown / Union Square' 'Tenderloin' 'Mission'
 'Bayview']
['South of Market' 'Tenderloin' 'Downtown / Union Square' 'Bernal Heights'
 'Marina']
['South of Market' 'Mission' 'Tenderloin' 'Downtown / Union Square'
 'Bayview']
['Downtown / Union Square' 'South of Market' 'Potrero Hill' 'Mission'
 'Outer Sunset']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Potrero Hill']
['Tenderloin' 'Potrero Hill' 'Downtown / Union Square' 'South of Market'
 'Bayview']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Potrero Hill']
['South of Market' 'Mission' 'Downtown / Union Square' 'Tenderloin'
 'Inner Richmond']
['South of Market' 'Downtown / Union Square' 'Tenderloin' 'Mission'
 'Bayview']
['Downtown / Union Square' 'South of Market' 'Tenderloin' 'Bayview'
 'Mission']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Bayview']
['South of Market' 'Mission' 'Tenderloin' 'Downtown / Union Square'
 'Bayview']
['Sou

['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Potrero Hill']
['Tenderloin' 'South of Market' 'Downtown / Union Square' 'Bayview'
 'Western Addition']
['South of Market' 'Downtown / Union Square' 'Mission' 'Tenderloin'
 'Potrero Hill']
['Mission' 'Tenderloin' 'South of Market' 'Civic Center'
 'Downtown / Union Square']
['South of Market' 'Downtown / Union Square' 'Tenderloin' 'Mission'
 'Western Addition']
['South of Market' 'Western Addition' 'Civic Center' 'Financial District'
 'Tenderloin']
['South of Market' 'Mission' 'Tenderloin' 'Downtown / Union Square'
 'Western Addition']
['South of Market' 'Downtown / Union Square' 'Pacific Heights' 'Mission'
 'Bayview']
['South of Market' 'Mission' 'Downtown / Union Square' 'Tenderloin'
 'Potrero Hill']
['South of Market' 'Downtown / Union Square' 'Civic Center' 'Tenderloin'
 'Western Addition']
['South of Market' 'Mission' 'Downtown / Union Square' 'Tenderloin'
 'Western Addition']
['Tenderloin' 'Bayview' 'Northern Wa

['Mission' 'Tenderloin' 'South of Market' 'Downtown / Union Square'
 'Western Addition']
['Mission' 'Tenderloin' 'South of Market' 'Northern Waterfront'
 'Hunters Point']
['Mission' 'Tenderloin' 'South of Market' 'Downtown / Union Square'
 'Western Addition']
['Tenderloin' 'Downtown / Union Square' 'South of Market' 'Mission'
 'Potrero Hill']
['Mission' 'Tenderloin' 'South of Market' 'Downtown / Union Square'
 'Western Addition']
['Mission' 'Tenderloin' 'South of Market' 'Western Addition' 'Mission Bay']
['Mission' 'Tenderloin' 'South of Market' 'Downtown / Union Square'
 'Western Addition']
['Mission' 'Downtown / Union Square' 'Civic Center' 'Tenderloin'
 'South of Market']
['Mission' 'Tenderloin' 'Downtown / Union Square' 'South of Market'
 'Western Addition']
['Tenderloin' 'South of Market' 'Bayview' 'Marina' 'Mission']


KeyboardInterrupt: 

<h3> Testing starts here <h3>

<h3> Test open_file <h3>

In [1153]:
def test_open_file(expected_length,expected_columns):
    df = open_file()
    assert len(df) == expected_length, "Data frame length not as expected."
    actual_columns = df.columns    
    assert len(actual_columns) == len(expected_columns), "Actual columns not as expected."
    for i in range(len(expected_columns)):
        assert expected_columns[i] == actual_columns[i], "Expected column " + expected_columns[i] + " but got " + actual_columns[i]
    print("All tests completed successfully")

In [1154]:
def get_all_columns():
    return ['Reports 1 day ago','Reports 2 days ago','Reports 3 days ago','Reports 4 days ago','Reports 5 days ago',
                    'Reports 6 days ago','Reports 7 days ago','Reports 14 days ago','Reports 30 days ago',
                    'Reports 365 days ago','Last 7 days reports','Last 14 days reports','Last 28 days reports',
                    'Number of businesses','Businesses 1 day ago','Businesses 2 days ago','Businesses 3 days ago',
                    'Businesses 4 days ago','Businesses 5 days ago','Businesses 6 days ago','Businesses 7 days ago',
                    'Businesses 14 days ago','Businesses 30 days ago','Businesses 365 days ago','Number of closures',
                    'Closures 1 day ago','Closures 2 days ago','Closures 3 days ago','Closures 4 days ago',
                    'Closures 5 days ago','Closures 6 days ago','Closures 7 days ago','Closures 14 days ago',
                    'Closures 30 days ago','Closures 365 days ago','Last 7 days closures','Last 14 days closures',
                    'Last 28 days closures','Number of openings','Openings 1 day ago','Openings 2 days ago',
                    'Openings 3 days ago','Openings 4 days ago','Openings 5 days ago','Openings 6 days ago',
                    'Openings 7 days ago','Openings 14 days ago','Openings 30 days ago','Openings 365 days ago',
                    'Last 7 days openings','Last 14 days openings','Last 28 days openings','Alamo Square','Anza Vista',
                    'Apparel City','Aquatic Park / Ft. Mason','Balboa Terrace','Bayview','Bernal Heights','Bret Harte',
                    'Buena Vista','Candlestick Point SRA','Castro','Cathedral Hill','Cayuga','Central Waterfront','Chinatown',
                    'Civic Center','Clarendon Heights','Cole Valley','Corona Heights','Cow Hollow','Crocker Amazon',
                    'Diamond Heights','Dogpatch','Dolores Heights','Downtown / Union Square','Duboce Triangle','Eureka Valley',
                    'Excelsior','Fairmount','Financial District','Fishermans Wharf','Forest Hill','Forest Knolls','Glen Park',
                    'Golden Gate Heights','Golden Gate Park','Haight Ashbury','Hayes Valley','Holly Park','Hunters Point',
                    'India Basin','Ingleside','Ingleside Terraces','Inner Richmond','Inner Sunset','Japantown','Laguna Honda',
                    'Lake Street','Lakeshore','Laurel Heights / Jordan Park','Lincoln Park / Ft. Miley','Little Hollywood',
                    'Lone Mountain','Lower Haight','Lower Nob Hill','Lower Pacific Heights','Marina','McLaren Park',
                    'Merced Heights','Merced Manor','Midtown Terrace','Mint Hill','Miraloma Park','Mission','Mission Bay',
                    'Mission Dolores','Mission Terrace','Monterey Heights','Mt. Davidson Manor','Nob Hill','Noe Valley',
                    'North Beach','Northern Waterfront','Oceanview','Outer Mission','Outer Richmond','Outer Sunset',
                    'Pacific Heights','Panhandle','Parkmerced','Parkside','Parnassus Heights','Peralta Heights',
                    'Polk Gulch','Portola','Potrero Hill','Presidio Heights','Presidio National Park','Presidio Terrace',
                    'Produce Market','Rincon Hill','Russian Hill','Seacliff','Sherwood Forest','Showplace Square',
                    'Silver Terrace','South Beach','South of Market','St. Francis Wood','St. Marys Park','Stonestown',
                    'Sunnydale','Sunnyside','Sutro Heights','Telegraph Hill','Tenderloin','Treasure Island','Union Street',
                    'University Mound','Upper Market','Visitacion Valley','West Portal','Western Addition',
                    'Westwood Highlands','Westwood Park','Yerba Buena Island','Friday','Saturday','Sunday','Thursday',
                    'Tuesday','Wednesday','Todays Reports','Date','Neighborhood']

In [1155]:
def get_all_neighbourhoods():
    return ['Alamo Square','Anza Vista', 'Apparel City','Aquatic Park / Ft. Mason','Ashbury Heights','Balboa Terrace','Bayview',
            'Bernal Heights','Bret Harte','Buena Vista','Candlestick Point SRA','Castro','Cathedral Hill','Cayuga',
            'Central Waterfront','Chinatown','Civic Center','Clarendon Heights','Cole Valley','Corona Heights','Cow Hollow',
            'Crocker Amazon','Diamond Heights','Dogpatch','Dolores Heights','Downtown / Union Square','Duboce Triangle',
            'Eureka Valley','Excelsior','Fairmount','Financial District','Fishermans Wharf','Forest Hill','Forest Knolls',
            'Glen Park','Golden Gate Heights','Golden Gate Park','Haight Ashbury','Hayes Valley','Holly Park','Hunters Point',
            'India Basin','Ingleside','Ingleside Terraces','Inner Richmond','Inner Sunset','Japantown','Laguna Honda',
            'Lake Street','Lakeshore','Laurel Heights / Jordan Park','Lincoln Park / Ft. Miley','Little Hollywood',
            'Lone Mountain','Lower Haight','Lower Nob Hill','Lower Pacific Heights','Marina','McLaren Park',
            'Merced Heights','Merced Manor','Midtown Terrace','Mint Hill','Miraloma Park','Mission','Mission Bay',
            'Mission Dolores','Mission Terrace','Monterey Heights','Mt. Davidson Manor','Nob Hill','Noe Valley',
            'North Beach','Northern Waterfront','Oceanview','Outer Mission','Outer Richmond','Outer Sunset',
            'Pacific Heights','Panhandle','Parkmerced','Parkside','Parnassus Heights','Peralta Heights',
            'Polk Gulch','Portola','Potrero Hill','Presidio Heights','Presidio National Park','Presidio Terrace',
            'Produce Market','Rincon Hill','Russian Hill','Seacliff','Sherwood Forest','Showplace Square',
            'Silver Terrace','South Beach','South of Market','St. Francis Wood','St. Marys Park','Stonestown',
            'Sunnydale','Sunnyside','Sutro Heights','Telegraph Hill','Tenderloin','Treasure Island','Union Street',
            'University Mound','Upper Market','Visitacion Valley','West Portal','Western Addition',
            'Westwood Highlands','Westwood Park','Yerba Buena Island']

In [1156]:
def get_all_ones():
    all_ones = []
    for i in range(0,117):
        all_ones.append(1)
    return all_ones

In [1157]:
#test successful load
expected_columns = get_all_columns()
DEFAULT_FILE_PATH = "tuning_test_data"
expected_length = 819
test_open_file(expected_length,expected_columns)

All tests completed successfully


In [1158]:
#test failed load
expected_columns = []
DEFAULT_FILE_PATH = "incorrect_path"
expected_length = 0
test_open_file(expected_length,expected_columns)

File not found
All tests completed successfully


<h3> Test get_hotspots <h3>

In [1159]:
def test_get_hotspots(test_model_name,test_feature_name,calendar_date, expected_y_actual, expected_actual_neighbourhoods,
                     expected_y_predict, expected_prediction_neighbourhoods):
    data = open_file()
    y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods = get_hotspots(data, 
                                                                                     test_model_name, 
                                                                                     test_feature_name,
                                                                                     calendar_date)
    for i in range(0,len(y_actual)):
        assert y_actual[i] == expected_y_actual[i], "Y actual not as expected."
    for i in range(0,len(expected_y_actual)):
        assert expected_y_actual[i] == y_actual[i], "Y actual not as expected."
        
    for i in range(0,len(y_predict)):
        assert y_predict[i] == expected_y_predict[i], "Y actual not as expected."
    for i in range(0,len(expected_y_predict)):
        assert expected_y_predict[i] == y_predict[i], "Y actual not as expected."   
    for i in range(0,len(actual_neighbourhoods)):
        assert actual_neighbourhoods[i] == expected_actual_neighbourhoods[i], "Actual neighbourhoods not as expected."
    for i in range(0,len(expected_actual_neighbourhoods)):
        assert expected_actual_neighbourhoods[i] == actual_neighbourhoods[i], "Actual neighbourhoods not as expected."    
    for i in range(0,len(prediction_neighbourhoods)):
        assert prediction_neighbourhoods[i] == expected_prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."  
    for i in range(0,len(expected_prediction_neighbourhoods)):
        assert expected_prediction_neighbourhoods[i] == prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."
    print("Tests completed successfully")

In [1160]:
DEFAULT_FILE_PATH = "tuning_test_data"
test_model_name = "Tuning Template Test"
test_model_file_tag = "tuning_template_test_model"
test_feature_file_tag = "arbitrary_name"
test_feature_name = "Arbitrary Feature Name"
test_feature_features = ['Reports 1 day ago', 'Reports 2 days ago', 'Reports 3 days ago',
                       'Reports 4 days ago', 'Reports 5 days ago', 'Reports 6 days ago',
                      'Reports 7 days ago','Reports 14 days ago','Reports 30 days ago','Reports 365 days ago']
FEATURES[test_feature_name] = test_feature_features
FEATURE_FILE_TAGS[test_feature_name] = test_feature_file_tag
FEATURE_NAMES_BY_FILE_TAG[test_feature_file_tag] = test_feature_name
MODEL_FILE_TAGS[test_model_name] = test_model_file_tag
calendar_date = "05/01/2021"

In [1161]:
expected_y_actual = [5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
expected_actual_neighbourhoods = ['Downtown / Union Square', 'Bernal Heights', 'Western Addition', 'Portola',
 'Bayview', 'Eureka Valley', 'South Beach', 'Lincoln Park / Ft. Miley',
 'Hunters Point', 'Potrero Hill', 'Presidio Heights', 'Cow Hollow',
 'Russian Hill', 'Cole Valley', 'Mission Terrace', 'Hayes Valley',
 'Stonestown', 'Central Waterfront', 'Northern Waterfront', 'Buena Vista',
 'Visitacion Valley', 'North Beach', 'Forest Hill', 'South of Market',
 'Haight Ashbury', 'Golden Gate Park', 'Yerba Buena Island',
 'Golden Gate Heights', 'Ingleside Terraces', 'Inner Richmond',
 'Inner Sunset', 'Lake Street', 'Lakeshore', 'Ingleside', 'Diamond Heights',
 'Glen Park', 'Forest Knolls', 'Financial District', 'Excelsior',
 'Duboce Triangle', 'Crocker Amazon', 'Corona Heights', 'Clarendon Heights',
 'Civic Center', 'Candlestick Point SRA', 'Balboa Terrace', 'Anza Vista',
 'Laurel Heights / Jordan Park', 'Merced Manor', 'Little Hollywood',
 'St. Francis Wood', 'Panhandle', 'Parkside', 'Presidio National Park',
 'Seacliff', 'Sherwood Forest', 'Silver Terrace', 'Sunnyside', 'Outer Sunset',
 'Telegraph Hill', 'Tenderloin', 'Treasure Island', 'Upper Market',
 'West Portal', 'Westwood Highlands', 'Pacific Heights', 'Outer Richmond',
 'Lone Mountain', 'Mission', 'Lower Pacific Heights', 'Marina',
 'Merced Heights', 'Westwood Park', 'Midtown Terrace', 'Miraloma Park',
 'Mission Bay', 'Outer Mission', 'Mission Dolores', 'Monterey Heights',
 'Mt. Davidson Manor', 'Nob Hill', 'Noe Valley', 'Oceanview', 'Alamo Square']
expected_y_predict = expected_y_actual
expected_prediction_neighbourhoods = expected_actual_neighbourhoods
test_get_hotspots(test_model_name,test_feature_name,calendar_date, expected_y_actual, expected_actual_neighbourhoods,
                     expected_y_predict, expected_prediction_neighbourhoods)

Tests completed successfully


<h3> Test load_model <h3>

In [1162]:
def test_load_model(x_data, y_data, neighbourhoods_data, model_key, features_key):
    y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods = load_model(x_data, 
                                                                                       y_data, 
                                                                                       neighbourhoods_data,
                                                                                       model_key, 
                                                                                       features_key)
    for i in range(0,len(y_actual)):
        assert y_actual[i] == expected_y_actual[i], "Y actual not as expected."
    for i in range(0,len(expected_y_actual)):
        assert expected_y_actual[i] == y_actual[i], "Y actual not as expected."   
    for i in range(0,len(y_predict)):
        assert y_predict[i] == expected_y_predict[i], "Y actual not as expected."
    for i in range(0,len(expected_y_predict)):
        assert expected_y_predict[i] == y_predict[i], "Y actual not as expected."   
    for i in range(0,len(actual_neighbourhoods)):
        assert actual_neighbourhoods[i] == expected_actual_neighbourhoods[i], "Actual neighbourhoods not as expected."
    for i in range(0,len(expected_actual_neighbourhoods)):
        assert expected_actual_neighbourhoods[i] == actual_neighbourhoods[i], "Actual neighbourhoods not as expected."    
    for i in range(0,len(prediction_neighbourhoods)):
        assert prediction_neighbourhoods[i] == expected_prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."  
    for i in range(0,len(expected_prediction_neighbourhoods)):
        assert expected_prediction_neighbourhoods[i] == prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."
    print("Tests completed successfully")

In [1163]:
all_ones = get_all_ones()
x_columns = ['Reports 1 day ago','Reports 2 days ago','Reports 3 days ago','Reports 4 days ago',
                      'Reports 5 days ago','Reports 6 days ago','Reports 7 days ago','Reports 14 days ago',
                      'Reports 30 days ago','Reports 365 days ago']
x_data = pd.DataFrame()
for col in x_columns:
    x_data[col] = all_ones
y_data = pd.DataFrame()
y_data['Todays Reports'] = all_ones
neighbourhoods_data = pd.DataFrame()
neighbourhoods_data['Neighborhood'] = get_all_neighbourhoods()
model_key = 'Tuning Template Test'
features_key = 'Arbitrary Feature Name'
test_load_model(x_data, y_data, neighbourhoods_data, model_key, features_key)

Tests completed successfully


<h3> Test make_prediction <h3>

In [1164]:
def test_make_prediction(model_path, x_data, y_data, neighbourhoods_data):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods = make_prediction(model,
                                                                                            x_data, 
                                                                                            y_data, 
                                                                                            neighbourhoods_data)
    for i in range(0,len(y_actual)):
        assert y_actual[i] == expected_y_actual[i], "Y actual not as expected."
    for i in range(0,len(expected_y_actual)):
        assert expected_y_actual[i] == y_actual[i], "Y actual not as expected."   
    for i in range(0,len(y_predict)):
        assert y_predict[i] == expected_y_predict[i], "Y actual not as expected."
    for i in range(0,len(expected_y_predict)):
        assert expected_y_predict[i] == y_predict[i], "Y actual not as expected."   
    for i in range(0,len(actual_neighbourhoods)):
        assert actual_neighbourhoods[i] == expected_actual_neighbourhoods[i], "Actual neighbourhoods not as expected."
    for i in range(0,len(expected_actual_neighbourhoods)):
        assert expected_actual_neighbourhoods[i] == actual_neighbourhoods[i], "Actual neighbourhoods not as expected."    
    for i in range(0,len(prediction_neighbourhoods)):
        assert prediction_neighbourhoods[i] == expected_prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."  
    for i in range(0,len(expected_prediction_neighbourhoods)):
        assert expected_prediction_neighbourhoods[i] == prediction_neighbourhoods[i], "Predicted neighbourhoods not as expected."
    print("Tests completed successfully")

In [1165]:
model_path = "CrimeGUI/Models/tuning_template_test_model_arbitrary_name"
test_make_prediction(model_path,x_data, y_data, neighbourhoods_data)

Tests completed successfully


<h3> Test get non negative value <h3>

In [1166]:
def test_get_non_negative_value(nums_in,expected_outs):
    for i in range(0,len(nums_in)):
        result = get_non_negative_value(nums_in[i])
        assert result == expected_outs[i], "Returned value not as expected"
    print("Tests completed successfully.")

In [1167]:
nums_in = [-10,-2,-1,0,1,2,10]
expected_outs = [0,0,0,0,1,2,10]
test_get_non_negative_value(nums_in,expected_outs)

Tests completed successfully.


<h3> Test merge sub neighbourhoods <h3>

In [1168]:
def test_merge_sub_neighbourhoods(y_data_in,y_predict_in,neighbourhoods_data_in, expected_y_actual, expected_y_predict,
                                 expected_neighbourhoods_data):
    y_actual,y_predict,neighbourhoods_data = merge_sub_neighbourhoods(y_data_in,y_predict_in,neighbourhoods_data_in)
    neighbourhoods_data.reset_index(drop=True, inplace=True)
    for i in range(0,len(y_actual)):
        assert y_actual[i] == expected_y_actual[i], "Y actual not as expected."
    for i in range(0,len(expected_y_actual)):
        assert expected_y_actual[i] == y_actual[i], "Y actual not as expected."   
    for i in range(0,len(y_predict)):
        assert y_predict[i] == expected_y_predict[i], "Y actual not as expected."
    for i in range(0,len(expected_y_predict)):
        assert expected_y_predict[i] == y_predict[i], "Y actual not as expected." 
    for i in range(0,len(neighbourhoods_data)):
        assert neighbourhoods_data['Neighborhood'][i] == expected_neighbourhoods_data[i], "Neighbourhoods not as expected."
    for i in range(0,len(expected_neighbourhoods_data)):
        assert expected_neighbourhoods_data[i] == neighbourhoods_data['Neighborhood'][i], "Neighbourhoods not as expected."   
    print("Tests completed successfully")

In [1169]:
y_data_in = pd.DataFrame()
y_data_in['Todays Reports'] = get_all_ones()
y_predict_in = all_ones
neighbourhoods_data_in = pd.DataFrame()
neighbourhoods_data_in['Neighborhood'] = get_all_neighbourhoods()
expected_y_actual = [1, 1, 1, 3, 4, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 5, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 
                     2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 2, 2, 1, 2, 1, 1, 1, 2,
                     2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1]
expected_y_predict = expected_y_actual
expected_neighbourhoods_data = ['Alamo Square', 'Anza Vista', 'Balboa Terrace', 'Bayview', 'Bernal Heights', 'Buena Vista', 
                                'Candlestick Point SRA', 'Central Waterfront', 'Civic Center', 'Clarendon Heights', 
                                'Cole Valley', 'Corona Heights', 'Cow Hollow', 'Crocker Amazon', 'Diamond Heights', 
                                'Downtown / Union Square', 'Duboce Triangle', 'Eureka Valley', 'Excelsior', 
                                'Financial District', 'Forest Hill', 'Forest Knolls', 'Glen Park', 'Golden Gate Heights', 
                                'Golden Gate Park', 'Haight Ashbury', 'Hayes Valley', 'Hunters Point', 'Ingleside', 
                                'Ingleside Terraces', 'Inner Richmond', 'Inner Sunset', 'Lake Street', 'Lakeshore', 
                                'Laurel Heights / Jordan Park', 'Lincoln Park / Ft. Miley', 'Little Hollywood', 
                                'Lone Mountain', 'Lower Pacific Heights', 'Marina', 'Merced Heights', 'Merced Manor', 
                                'Midtown Terrace', 'Miraloma Park', 'Mission', 'Mission Bay', 'Mission Dolores', 
                                'Mission Terrace', 'Monterey Heights', 'Mt. Davidson Manor', 'Nob Hill', 'Noe Valley', 
                                'North Beach', 'Northern Waterfront', 'Oceanview', 'Outer Mission', 'Outer Richmond', 
                                'Outer Sunset', 'Pacific Heights', 'Panhandle', 'Parkside', 'Portola', 'Potrero Hill', 
                                'Presidio Heights', 'Presidio National Park', 'Russian Hill', 'Seacliff', 'Sherwood Forest', 
                                'Silver Terrace', 'South Beach', 'South of Market', 'St. Francis Wood', 'Stonestown', 
                                'Sunnyside', 'Telegraph Hill', 'Tenderloin', 'Treasure Island', 'Upper Market', 
                                'Visitacion Valley', 'West Portal', 'Western Addition', 'Westwood Highlands', 'Westwood Park', 
                                'Yerba Buena Island']
test_merge_sub_neighbourhoods(y_data_in,y_predict_in,neighbourhoods_data_in, expected_y_actual, expected_y_predict,
                                 expected_neighbourhoods_data)

Tests completed successfully


<h3> Test calculate accuracy <h3>

In [1170]:
def test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods, expected_scores
                            ,num_hotspots):
    
    result = calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods,num_hotspots)
    #returns [sensitivity, specificity, precision, f1, mcc, misclassification_severity]
    print(result)
    assert result [0] == expected_scores [0], "Sensitivity not as expected."
    assert result [1] == expected_scores [1], "Specificity not as expected."
    assert result [2] == expected_scores [2], "Precision not as expected."
    assert result [3] == expected_scores [3], "F1 not as expected."
    assert result [4] == expected_scores [4], "MCC not as expected."
    assert result [5] == expected_scores [5], "Misclassification severity not as expected."
    print ("All tests completed successfully")

In [1171]:
def get_x_neighbourhoods(x, reverse):
    neighbourhoods = []
    for i in range(0,x):
        neighbourhoods.append('Neighbourhood ' + str(i))
    if reverse:
        neighbourhoods.reverse()
    return neighbourhoods

In [1172]:
data = open_file()
test_model_name = "Tuning Template Test"
test_feature_name = "Arbitrary Feature Name"
calendar_date = "05/01/2021"
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods = get_hotspots(data, 
                                                                                     test_model_name, 
                                                                                     test_feature_name,
                                                                                     calendar_date)
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, prediction_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
All tests completed successfully


In [1173]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1174]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1175]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1176]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1177]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(one_to_ten))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1178]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1179]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(one_to_ten))
y_predict = np.ndarray((10,),buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1180]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(one_to_ten))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1181]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(one_to_ten))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1182]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(one_to_ten))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 10
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1, 1.0, 1.0, 1, 0.0]
All tests completed successfully


In [1183]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
All tests completed successfully


In [1184]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [0.8888888888888888,0,0.8888888888888888,0.88889,-0.11111,0.16666666666666666]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[0.8888888888888888, 0.0, 0.8888888888888888, 0.88889, -0.11111, 0.16666666666666666]
All tests completed successfully


In [1185]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [0.8888888888888888,0,0.8888888888888888,0.88889,-0.11111,0.16666666666666666]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[0.8888888888888888, 0.0, 0.8888888888888888, 0.88889, -0.11111, 0.16666666666666666]
All tests completed successfully


In [1186]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
All tests completed successfully


In [1187]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
All tests completed successfully


In [1188]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = False), dtype=object)
expected_scores = [0.8888888888888888,0,0.8888888888888888,0.88889,-0.11111,0.16666666666666666]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[0.8888888888888888, 0.0, 0.8888888888888888, 0.88889, -0.11111, 0.16666666666666666]
All tests completed successfully


In [1189]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,),buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = False), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [0.8888888888888888,0,0.8888888888888888,0.88889,-0.11111,0.16666666666666666]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[0.8888888888888888, 0.0, 0.8888888888888888, 0.88889, -0.11111, 0.16666666666666666]
All tests completed successfully


In [1190]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
one_to_ten = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_actual = np.ndarray((10,), buffer = np.array(ten_to_one))
y_predict = np.ndarray((10,), buffer = np.array(one_to_ten))
actual_neighbourhoods = np.array(get_x_neighbourhoods(len(y_actual), reverse = True), dtype=object)
predicted_neighbourhoods = np.array(get_x_neighbourhoods(len(y_predict), reverse = True), dtype=object)
expected_scores = [1,1,1,1,1,0]
num_hotspots = 9
test_calculate_accuracy(y_actual, actual_neighbourhoods, y_predict, predicted_neighbourhoods, expected_scores, num_hotspots)

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
All tests completed successfully


<h3> Test determine hotspots <h3>

In [1191]:
def test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result):
    result = determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots)[0]
    for i in range (0,len(expected_result)):
        assert expected_result[i] in result, "Hotspots not as expected"
    print("All tests completed successfully")

In [1192]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
num_hotspots = 10
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1193]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
num_hotspots = 10
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1194]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
expected_result = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
num_hotspots = 10
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1195]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
expected_result = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
num_hotspots = 10
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1196]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(9, reverse = False), dtype=object)
num_hotspots = 9
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1197]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(9, reverse = False), dtype=object)
num_hotspots = 9
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1198]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
expected_result = np.array(['Neighbourhood 9','Neighbourhood 8', 'Neighbourhood 7', 'Neighbourhood 6', 'Neighbourhood 5' ,
                   'Neighbourhood 4', 'Neighbourhood 3', 'Neighbourhood 2', 'Neighbourhood 1'], dtype=object)
print(expected_result)
print(type(expected_result))
num_hotspots = 9
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

['Neighbourhood 9' 'Neighbourhood 8' 'Neighbourhood 7' 'Neighbourhood 6'
 'Neighbourhood 5' 'Neighbourhood 4' 'Neighbourhood 3' 'Neighbourhood 2'
 'Neighbourhood 1']
<class 'numpy.ndarray'>
All tests completed successfully


In [1199]:
ten_to_one = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(ten_to_one))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
expected_result = np.array(['Neighbourhood 9','Neighbourhood 8', 'Neighbourhood 7', 'Neighbourhood 6', 'Neighbourhood 5' ,
                   'Neighbourhood 4', 'Neighbourhood 3', 'Neighbourhood 2', 'Neighbourhood 1'], dtype=object)
num_hotspots = 9
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1200]:
five_to_zero = [5.0,4.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0]
y_actual = np.ndarray((10,),buffer = np.array(five_to_zero))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(5, reverse = False), dtype=object)
num_hotspots = 5
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1201]:
five_to_zero = [5.0,4.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0]
y_actual = np.ndarray((10,),buffer = np.array(five_to_zero))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(5, reverse = False), dtype=object)
num_hotspots = 5
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1202]:
five_to_zero = [5.0,4.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0]
y_actual = np.ndarray((10,),buffer = np.array(five_to_zero))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = True), dtype=object)
expected_result = np.array(['Neighbourhood 9', 'Neighbourhood 8', 'Neighbourhood 7', 'Neighbourhood 6',
                            'Neighbourhood 5'], dtype=object)
num_hotspots = 5
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


In [1203]:
five_to_zero = [5.0,4.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0]
y_actual = np.ndarray((10,),buffer = np.array(five_to_zero))
actual_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
prediction_neighbourhoods = np.array(get_x_neighbourhoods(10, reverse = False), dtype=object)
expected_result = np.array(get_x_neighbourhoods(5, reverse = True), dtype=object)
num_hotspots = 5
test_determine_hotspots(y_actual, actual_neighbourhoods, prediction_neighbourhoods,num_hotspots,expected_result)

All tests completed successfully


<h3> Test calculate_standard_scores <h3>

In [1216]:
def test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected):
    num_predictions = len(predicted_hotspots)
    expected_result = np.ndarray((5,),buffer = np.array(expected))
    result = calculate_standard_scores(num_predictions, predicted_hotspots, actual_hotspots, total_predictions)
    assert len(expected_result) == len(result), "Scores not as expected"
    for i in range (0,len(expected_result)):
        assert expected_result[i] == result[i], "Scores not as expected"
    print("All tests completed successfully")

In [1217]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
total_predictions = 5
expected_result = [1.0,1.0,1.0,1.0,1.0]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1218]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
total_predictions = 10
expected_result = [1.0,1.0,1.0,1.0,1.0]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1219]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 6']
total_predictions = 5
expected_result = [0.8,1,0.8,0.8,1.0]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1220]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 6']
total_predictions = 10
expected_result = [0.8,0.8,0.8,0.8,0.6]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1221]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 6']
total_predictions = 15
expected_result = [0.8,0.9,0.8,0.8,0.7]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1222]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10']
total_predictions = 5
expected_result = [0.0,1.0,0.0,0.0,1.0]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1223]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10']
total_predictions = 10
expected_result = [0.0,0.0,0.0,0.0,-1.0]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


In [1224]:
predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10']
total_predictions = 15
expected_result = [0.0,0.5,0.0,0.0,-0.5]
test_calculate_standard_scores(predicted_hotspots, actual_hotspots, total_predictions,expected_result)

All tests completed successfully


<h3> Test get_missed_incidents <h3>

In [1261]:
def test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result):
    num_predictions = len(predicted_hotspots)
    
    
    actual_result = get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, num_predictions, 
                                  actual_neighbourhoods)

    assert actual_result[0] == expected_result[0], "Result not as expected."
    print("All tests completed successfully.")

In [1262]:
actuals = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']


expected_result = [0]
test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.


In [1263]:
actuals = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 6']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']

actual_hotspot_incidents = 10+9+8+7+6
predicted_hotspot_incidents = 10+9+8+7+5
missed = actual_hotspot_incidents - predicted_hotspot_incidents
expected_result = [missed/actual_hotspot_incidents]

test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.


In [1264]:
actuals = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 7']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']

actual_hotspot_incidents = 10+9+8+7+6
predicted_hotspot_incidents = 10+9+8+7+4
missed = actual_hotspot_incidents - predicted_hotspot_incidents
expected_result = [missed/actual_hotspot_incidents]

test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.


In [1265]:
actuals = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']

actual_hotspot_incidents = 10+9+8+7+6
predicted_hotspot_incidents = 1+2+3+4+5
missed = actual_hotspot_incidents - predicted_hotspot_incidents
expected_result = [missed/actual_hotspot_incidents]

test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.


In [1266]:
actuals = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 10','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']

actual_hotspot_incidents = 10+9+8+7+6
predicted_hotspot_incidents = 1+9+8+7+6
missed = actual_hotspot_incidents - predicted_hotspot_incidents
expected_result = [missed/actual_hotspot_incidents]

test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.


In [1267]:
actuals = [10.0,9.0,8.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0]
y_actual = np.ndarray((10,),buffer = np.array(actuals))
predicts = [10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0]
y_predict = np.ndarray((10,),buffer = np.array(predicts))
actual_neighbourhoods = np.array(['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5',
                                  'Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10'], 
                                 dtype=object)


predicted_hotspots = ['Neighbourhood 6','Neighbourhood 7','Neighbourhood 8','Neighbourhood 9','Neighbourhood 10']
actual_hotspots = ['Neighbourhood 1','Neighbourhood 2','Neighbourhood 3','Neighbourhood 4','Neighbourhood 5']

actual_hotspot_incidents = 10+9+8+7+6
predicted_hotspot_incidents = 0+0+0+0+0
missed = actual_hotspot_incidents - predicted_hotspot_incidents
expected_result = [missed/actual_hotspot_incidents]

test_get_missed_incidents(predicted_hotspots, actual_hotspots, y_actual, y_predict, actual_neighbourhoods,expected_result)

All tests completed successfully.
